In [ ]:
# Data fetching library
import requests as req
# used below: 'res' stands for 'response'

# Data manipulation libraries
import pandas as pd
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

# Data visualization libraries
import folium

# Find information about all data on the Resource Watch API, format as DataFrame

In [ ]:
# Base URL for getting dataset metadata from RW API
# Metadata = Data that describes Data 
url = "https://api.resourcewatch.org/v1/dataset?sort=slug,-provider,userId&status=saved&includes=metadata,vocabulary,widget,layer"

# page[size] tells the API the maximum number of results to send back
# There are currently between 200 and 300 datasets on the RW API
payload = { "application":"rw", "page[size]": 1000}

# Request all datasets, and extract the data from the response
res = req.get(url, params=payload)
data = res.json()["data"]

#############################################################

### Convert the json object returned by the API into a pandas DataFrame
# Another option: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html
datasets_on_api = {}
for ix, dset in enumerate(data):
    atts = dset["attributes"]
    metadata = atts["metadata"]
    layers = atts["layer"]
    widgets = atts["widget"]
    tags = atts["vocabulary"]
    datasets_on_api[atts["name"]] = {
        "rw_id":dset["id"],
        "table_name":atts["tableName"],
        "provider":atts["provider"],
        "date_updated":atts["updatedAt"],
        "num_metadata":len(metadata),
        "metadata": metadata,
        "num_layers":len(layers),
        "layers": layers,
        "num_widgets":len(widgets),
        "widgets": widgets,
        "num_tags":len(tags),
        "tags":tags
    }

# Create the DataFrame, name the index, and sort by date_updated
# More recently updated datasets at the top
current_datasets_on_api = pd.DataFrame.from_dict(datasets_on_api, orient='index')
current_datasets_on_api.index.rename("Dataset", inplace=True)
current_datasets_on_api.sort_values(by=["date_updated"], inplace=True, ascending = False)

In [ ]:
# Simple search:
search_terms = ["flood", "fire"]
# View datasets on the Resource Watch API
dataset_matches = [name for name in current_datasets_on_api.index if any(term in name.lower() for term in search_terms)]

dataset_matches

In [ ]:
disaster_data = current_datasets_on_api.loc[dataset_matches]
disaster_data

# Floods

In [ ]:
disaster_data.loc["Floods News Reports", "layers"]

In [ ]:
rw_id = "a972a30d-f766-4d68-a85d-392538b4ccf9"
table_name = "flood_observatory"
#sql = "SELECT st_x(the_geom) as lon, st_y(the_geom) as lat FROM flood_observatory where to_date(date_began, 'DD-Mon-YY') > (CURRENT_DATE - interval '3 year')"
sql_explore = "Select * from flood_observatory"

query_base = "https://api.resourcewatch.org/v1/query/{}?sql={}"

query = query_base.format(rw_id, sql_explore)
#query = query_base.format(rw_id, sql)

res = req.get(query)

flood_data = res.json()["data"]

In [ ]:
len(flood_data)

In [ ]:
m = folium.Map(location=[0, 0], zoom_start=1, tiles='OpenStreetMap')
for point in flood_data:
    folium.Marker(location=[point["lat"], point["lon"]]).add_to(m)

m

## The embedded map does not show in GitHub

# Fires

In [ ]:
disaster_data.loc["VIIRS Active Fire -- Global", "layers"]

In [ ]:
import requests as req

rw_id = "20cc5eca-8c63-4c41-8e8e-134dcf1e6d76"
# table_name = "vnp14imgtdl_nrt_global_7d"

sql_with_date = "SELECT st_x(the_geom) as lon, st_y(the_geom) as lat FROM vnp14imgtdl_nrt_global_7d where to_date(substring(cast(acq_date as text), 1, 10), 'yyyy-mm-dd') > (CURRENT_DATE - interval '1 day')"

sql_with_date_count = "SELECT count(*) FROM vnp14imgtdl_nrt_global_7d where to_date(substring(cast(acq_date as text), 1, 10), 'yyyy-mm-dd') > (CURRENT_DATE - interval '1 day')"

sql_with_date = "SELECT acq_date FROM vnp14imgtdl_nrt_global_7d where to_date(substring(cast(acq_date as text), 1, 10), 'yyyy-mm-dd') > (CURRENT_DATE - interval '1 day')"
query1 = query_base.format(rw_id, sql_with_date)
query_base = "https://api.resourcewatch.org/v1/query/{}?sql={}"
payload = { "application":"rw", "page[size]": 1000000000}
#res1 = req.get(query1, params=payload)

sql = "SELECT st_x(the_geom) as lon, st_y(the_geom) as lat, acq_date as date FROM vnp14imgtdl_nrt_global_7d LIMIT {}"
limit = 9000
sql = sql.format(limit)

rw_id = "20cc5eca-8c63-4c41-8e8e-134dcf1e6d76"
query_base = "https://api.resourcewatch.org/v1/query/{}?sql={}"
query2 = query_base.format(rw_id, sql)

payload = { "application":"rw", "page[size]": 10000000}
res2 = req.get(query2, params=payload)

# Breaks
#res1 = req.get(query1, params=payload)
# Works


    
fire_data = res2.json()["data"]

In [ ]:
sql = "SELECT st_x(the_geom) as lon, st_y(the_geom) as lat, acq_date as date FROM vnp14imgtdl_nrt_global_7d LIMIT {}"
limit = 9000
sql = sql.format(limit)

rw_id = "20cc5eca-8c63-4c41-8e8e-134dcf1e6d76"
query_base = "https://api.resourcewatch.org/v1/query/{}?sql={}"
query2 = query_base.format(rw_id, sql)

payload = { "application":"rw", "page[size]": 10000000}
res2 = req.get(query2, params=payload)

fire_data = res2.json()["data"]

In [ ]:
len(fire_data)

In [ ]:
len(fire_data)

In [ ]:
rw_id = "a9e33aad-eece-4453-8279-31c4b4e0583f"
#table_name = "modis_c6_global_7d"

sql = "SELECT st_x(the_geom) as lon, st_y(the_geom) as lat, acq_date as date FROM modis_c6_global_7d LIMIT 9500"
query_base = "https://api.resourcewatch.org/v1/query/{}?sql={}"
query2 = query_base.format(rw_id, sql)
res2 = req.get(query2)
    
fire_data = res2.json()["data"]


In [ ]:
len(fire_data)

In [ ]:
### This approach didn't work for me, so I switched to below
# #http://bobrochel.blogspot.in/2010/11/bad-servers-chunked-encoding-and.html?showComment=1358777800048
# try:
#     res = req.get(query)
# except (httplib.IncompleteRead, e):
#     res = e.partial
    
    
# https://stackoverflow.com/questions/39116123/protocolerror-incompleteread-using-requests
partial = ""

response = req.get(query2, stream=True)

if response.ok:
    for block in response.iter_content(1024):
        print("ok")
        print(str(block))
        if not block:
            break
        partial += str(block)
        
### You have to adjust the end_index so that 
# the last character in partial is a }
end_index = -3
partial[2:end_index]

import json
fire_data = json.loads((partial[2:-3]+"]}").replace("\'b\'", ""))["data"]
len(fire_data)

## I've run this multiple times and get 389 each time

In [ ]:
len(fire_data)

In [ ]:
fire_data[0:10]

In [ ]:
m = folium.Map(location=[0, 0], zoom_start=1, tiles='OpenStreetMap')
for point in fire_data:
    folium.Marker(location=[point["lat"], point["lon"]]).add_to(m)

m

## The embedded map does not show in GitHub